In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# CELL 1 — Lens configuration input block
# =======================================
# Change ONLY this cell for each lens.

# 1. Redshifts
z_d = 0.7454
z_s = 1.789

# 2. Angular-diameter distances (from TDCOSMO tables)
D_d  = 1492.0 * u.Mpc
D_s  = 2120.0 * u.Mpc
D_ds = 1082.0 * u.Mpc

# ---- ADD THIS BLOCK (distance conversions + prefactor) ----
# Convert distances to meters
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

# Time-delay distance prefactor:  (1+z_d) * D_d * D_s / D_ds
time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)
# ------------------------------------------------------------

# 3. Einstein radius
theta_E_arcsec = 1.7017

# 4. Galaxy light-shape parameters (Sersic-like)
axis_ratio_q = 0.685
position_angle_phi = 24.0    # degrees

# 5. External shear
gamma_ext = 0.076
phi_gamma_deg = 73.0

# 6. External convergence
kappa_env = 0.065   # LOS environment mass sheet

# 7. Image positions A, B, C, D  (arcsec)
image_positions = {
    "A": ( +0.625, +0.220 ),
    "B": ( -0.685, -0.172 ),
    "C": ( -0.110, +0.735 ),
    "D": ( +0.115, -0.690 )
}

# 8. MGE Parameters (example for J1206)
# You can replace with literature MGE for other systems.
MGE_sigmas_arcsec = [0.04, 0.08, 0.15, 0.25, 0.40,
                     0.65, 1.00, 1.60, 2.50, 4.00]
MGE_amps          = [10.0, 8.0, 6.0, 4.5, 3.0,
                      2.0, 1.2, 0.6, 0.25, 0.10]

In [ ]:
# CELL 2 — Build grid & MGE model (does not change)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# CELL 5 — Evaluate ψ_SFH at image positions and compute Δt (does not change)

def psi_at(theta_x, theta_y):
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# Extract real images
psiA = psi_at(*image_positions["A"])
psiB = psi_at(*image_positions["B"])

delta_psi = abs(psiB - psiA)

delta_t_raw = (delta_psi * time_delay_distance_factor / c) / (24*3600)
delta_t_corr = delta_t_raw / (1 - kappa_env)

print(f"ψ_A = {psiA:.6e}")
print(f"ψ_B = {psiB:.6e}")
print(f"Δψ = {delta_psi:.6e}")
print(f"Raw SFH Δt = {delta_t_raw:.2f} days")
print(f"κ_env-corrected = {delta_t_corr:.2f} days")
print()
print("Observed: 112.8 ± 3 days")

ψ_A = 1.964580e-10
ψ_B = 1.785603e-10
Δψ = 1.789767e-11
Raw SFH Δt = 108.79 days
κ_env-corrected = 116.35 days

Observed: 112.8 ± 3 days
